In [1]:
import itertools

import matplotlib.pyplot as plt
import numpy as np

from stats import *
from utils import *
import visual as vi
from Inlier_Thresholder import Inlier_Thresholder
import scipy.io as spi
from time import time
import os
import seaborn as sns
from itertools import chain

# FUZZY VALUE OF THE CLUSTERING OF LMEDS

In [2]:
directory = 'C:/Users/carlo/IACV PROJECT/adelH'

# List all files in the directory
files = os.listdir(directory)

# Filter out only the .mat files
names = [file for file in files if file.endswith('.mat')]

# Load each .mat file
mat_data = {}
for file in names:
    file_path = os.path.join(directory, file)
    mat_data[file] = spi.loadmat(file_path)
    

In [3]:
# input parameter, how many files to analyse
number_of_data_to_analyse=len(names)


thresholds = [[] for i in range(number_of_data_to_analyse)]
res_matrices=[]
part_matrices=[]

H_lmeds=[]



for i in range(number_of_data_to_analyse):
    data=mat_data[names[i]]
    res_mat=build_residual_matrix(data,verbose=False)
    
    res_matrices.append(res_mat)
    part_matrices.append(build_partition_matrix(res_mat))
    
    res=compute_inliers_residual_curve(data)
    
    data=mat_data[names[i]]
    
    img1, img2 = data["img1"], data["img2"]

    outliers, models = vi.group_models(data)["outliers"], vi.group_models(data)["models"]
    
    points=extract_points(models,data)
    
    labels = points[3]
    
    points=points[0]
    
    H_models=[]
    
    for m in range(len(models)):
        src = points["src_points"][m]
        dst = points["dst_points"][m]
        
        H,_=verify_LMEDS_H(src,dst,verbose=False)
        
        H_models.append(H)
        
    H_lmeds.append(H_models)
        

    for j in range(len(res)):
        anomaly_detector=Inlier_Thresholder(res[j])

        labels, threshold=anomaly_detector.use_best_method(verbose=False)  # use best method among the statistical ones, best according to silhouette
        
        if len(np.unique(labels))!=1:
            thresholds[i].append(threshold)


In [4]:
fuzzy_sil=[]
for i in range(len(res_matrices)):
    fuzzy_sil.append(Fuzzy_silhouette(res_matrices[i],part_matrices[i]))
    
np.mean(fuzzy_sil)

0.9113758227051876

# GC RANSAC WITH RIGHT INLIER THRESHOLD

In [5]:
thresholds

[[3.9059483895348674, 5.061296450727099],
 [1.2727631044728074,
  1.3540355136313136,
  1.6518908412426505,
  1.314670265238886,
  1.110507308045321,
  0.7774561582355207],
 [4.47025515869321],
 [12.15950442095154, 4.922981679454178],
 [1.8931507520140534, 1.0025402084617272, 3.4689818487661435],
 [4.013263635193033, 3.4420158659309443],
 [10.65333665445521,
  1.4972973899491056,
  4.855841909636071,
  2.1346919352420777],
 [20.48936186639708,
  10.65333665445521,
  3.5880995829430957,
  4.855841909636071,
  19.1939351600231,
  2.2929432197981274,
  2.1346919352420777],
 [7.587707981890549, 2.3732627091400564],
 [2.9236960966399845, 2.621117722733934],
 [1.7893970318330898, 6.024208380026],
 [18.664565650574087, 2.9567515905492634, 6.3074591885583216],
 [2.6793874293083793, 3.841510567003268, 7.794031341459188],
 [2.9236960461703045, 1.4929407152901897],
 [2.629437652198048, 1.1390767648300542],
 [9.911076035281326],
 [4.151204581395456, 1.6038197688786329],
 [1.1990669463070318,
  3.6

In [6]:
# input parameter, how many files to analyse
number_of_data_to_analyse=len(names)



res_matrices=[]
part_matrices=[]
H_gc=[]

residuals=[]





for i in range(number_of_data_to_analyse):
    data=mat_data[names[i]]
    
    img1, img2 = data["img1"], data["img2"]

    outliers, models = vi.group_models(data)["outliers"], vi.group_models(data)["models"]
    
    points=extract_points(models,data)
    
    labels = points[3]
    
    points=points[0]
    
    H_models=[]
    
    H_res_models=[]
    
    for m in range(len(models)):
        src = points["src_points"][m]
        dst = points["dst_points"][m]
        
        H,_=verify_pygcransac_H(src,dst,img1,img2,threshold=thresholds[i][m],verbose=False)
        
        H_models.append(H)
        
        H_residual= residual_H1_wrt_H2(src,H,H_lmeds[i][m])
        
        H_res_models.append(H_residual)
        
    residuals.append(H_res_models)
        
    H_gc.append(H_models)
    
    
    res=build_residual_matrix(data, plot=False, verbose=False, method="gc-ransac" , threshold=thresholds[i])
    
    res_matrices.append(res)
    part_matrices.append(build_partition_matrix(res))

        
        

In [7]:
fuzzy_sil=[]
for i in range(len(res_matrices)):
    fuzzy_sil.append(Fuzzy_silhouette(res_matrices[i],part_matrices[i]))
    
np.mean(fuzzy_sil)

0.918209408831556

In [8]:
flattened_residuals=[r for s in residuals for c in s for r in c]
np.mean(flattened_residuals)

0.19857924698415041